# Techniques of High Performance Computing - Assignment 2

### Name: John Duffy

### Student Number: 19154676

# IMPORTANT NOTE


In [2]:
# Import libraries common to Questions 1 & 2.

import numpy as np
import pyopencl as cl

from scipy.sparse import csr_matrix, eye
from scipy.sparse.linalg import LinearOperator, cg

# Question 1

## OpenCL CSR Matrix-Vector Product

**IMPORTANT NOTE**

My MacBook Pro is equipped with an Intel Core i5 CPU and an Intel Iris Plus Graphics GPU as depicted below.

    cl.get_platforms()[0].get_devices()

    [<pyopencl.Device 'Intel(R) Core(TM) i5-8279U CPU @ 2.40GHz' on 'Apple' at 0xffffffff>,
     <pyopencl.Device 'Intel(R) Iris(TM) Plus Graphics 655' on 'Apple' at 0x1024500>]
     
To ensure a context is created using the CPU for Question 1 (so that AVX2 technology is available) the function cl.Context() is used as below.

    ctx = cl.Context(dev_type = cl.device_type.CPU)  # Tell OpenCL to use the CPU device.
    
The function cl.get_some_context() used in the course lecture notes is not specific enough (on my MacBook Pro at least) which then causes kernel build problems. This seems to be specific to macOS 10.14 Mojave.

### Program Description

This program...


### Optimisation Results

A class LinearOperatorBaseline was created to use as a baseline for measuring the performance gains through subsequnt use of OpenCL and AVX2.

In [3]:
# Define the class LinearOperatorBaseline for baseline performance measurements.

class LinearOperatorBaseline(LinearOperator):
    """
    This class... 
    """
    
    def __init__(self, data, indices, indptr):
        """
        """
        self.data = data
        self.indices = indices
        self.indptr = indptr
        self.shape = (len(indptr) - 1, len(indptr) - 1)  # Assume N x N.
    
    def _matvec(self, x):
        """
        """
        y = np.zeros(x.shape[0], dtype=np.float64)
        
        for i in range(self.shape[1]): 
            y[i] = np.dot(self.data[self.indptr[i]:self.indptr[i + 1]], x[self.indices[self.indptr[i]]:self.indptr[i + 1]])
            
        return y

In [4]:
# Create a test instance of LinearOperatorBaseline.

N = 1000

csr = eye((N), dtype=np.float64).tocsr()

linear_operator = LinearOperatorBaseline(csr.data, csr.indices, csr.indptr)

v = np.full((N), 5, dtype=np.float64)

In [5]:
#%%timeit

#linear_operator.matvec(v)

In [6]:
#%%timeit

#linear_operator * v

In [7]:
linear_operator.matvec(v)

array([5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5.

In [8]:
linear_operator * v

array([5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5.

In [9]:
# Define the class LinearOperatorOpenCL.

class LinearOperatorOpenCL(LinearOperator):
    """
    This class... 
    
    
    Note: This class assumes the CSR matrix is N x N and derives N from the length of indptr parameter.   
    """
    
    def __init__(self, data, indices, indptr):
        """
        This function...
        """
        
        self.shape = (len(indptr) - 1, len(indptr) - 1)
        
        self.ctx = cl.Context(dev_type = cl.device_type.CPU)
        
        self.queue = cl.CommandQueue(self.ctx, properties=cl.command_queue_properties.PROFILING_ENABLE)
        
        mf = cl.mem_flags
        
        # Copy the CSR matrix data to the device.
 
        self.device_global_data = cl.Buffer(self.ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = data)
        self.device_global_indices = cl.Buffer(self.ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = indices)       
        self.device_global_indptr = cl.Buffer(self.ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = indptr)

        # Reserve space on the device for the input and output vectors.
        
        self.device_global_v = cl.Buffer(self.ctx, mf.READ_ONLY, size=self.shape[0] * np.dtype(np.float64).itemsize)
        self.device_global_y = cl.Buffer(self.ctx, mf.WRITE_ONLY, size=self.shape[0] * np.dtype(np.float64).itemsize)

        self.program = cl.Program(self.ctx, """
        __kernel void matvec_opencl(
            __global const double *data,
            __global const double *indices,
            __global const double *indptr,
            __global const double *v,
            __global double *y
        )
        {
            double8 result[4] = {0.0, 0.0, 0.0, 0.0};
 
            int gid = get_global_id(0);
 
            y[gid] = 10 * v[gid]; 

        }
        """).build()

    
    def _matvec(self, host_v):
        """
        """

        cl.enqueue_copy(self.queue, self.device_global_v, host_v)

        self.program.matvec_opencl(self.queue, \
                                   (self.shape[0],), \
                                   (1,), \
                                   self.device_global_data, \
                                   self.device_global_indices, \
                                   self.device_global_indptr, \
                                   self.device_global_v, \
                                   self.device_global_y \
                                  )

        host_y = np.zeros((self.shape[0]), dtype=np.float64)

        cl.enqueue_copy(self.queue, host_y, self.device_global_y)
        
               
        #for i in range(self.shape[1]): 
        #y[i] = np.dot(self.data[self.indptr[i]:self.indptr[i + 1]], x[self.indices[self.indptr[i]]:self.indptr[i + 1]])
            
        return host_y


In [10]:
# Create a test instance of LinearOperatorOpenCL.

N = 16

csr = eye((N), dtype=np.float64).tocsr()

linear_operator = LinearOperatorOpenCL(csr.data, csr.indices, csr.indptr)

In [11]:
# Test the instance.

v = np.full((N), 33, dtype=np.float64)

linear_operator.matvec(v)

array([330., 330., 330., 330., 330., 330., 330., 330., 330., 330., 330.,
       330., 330., 330., 330., 330.])

In [12]:
#%%timeit

#linear_operator.matvec(v)

In [13]:
#%%timeit

#linear_operator * v

In [14]:
#linear_operator.matvec(v)

In [15]:
#linear_operator * v

# Question 2

## Solving a Poisson Problem with OpenCL

**IMPORTANT NOTE**

My MacBook Pro is equipped with an Intel Core i5 CPU and an Intel Iris Plus Graphics GPU as depicted below.

    cl.get_platforms()[0].get_devices()

    [<pyopencl.Device 'Intel(R) Core(TM) i5-8279U CPU @ 2.40GHz' on 'Apple' at 0xffffffff>,
     <pyopencl.Device 'Intel(R) Iris(TM) Plus Graphics 655' on 'Apple' at 0x1024500>]
     
To ensure a context is created using the GPU for Question 2 the function cl.Context() is used as below.

    ctx = cl.Context(dev_type = cl.device_type.GPU)  # Tell OpenCL to use the GPU device.
    
The function cl.get_some_context() used in the course lecture notes is not specific enough (on my MacBook Pro at least) which then causes kernel build problems. This seems to be specific to macOS 10.14 Mojave.

### Program Description

This program...

In [301]:
# Define the class StencilOperatorOpenCL.

class StencilOperatorOpenCL(LinearOperator):
    """
    This class... 
    """
    
    def __init__(self, m):
        """
        This function...
        """
        
        self.shape = (m * m, m * m)
        
        self.ctx = cl.Context(dev_type = cl.device_type.GPU)
        
        self.queue = cl.CommandQueue(self.ctx, properties=cl.command_queue_properties.PROFILING_ENABLE)
        
        mf = cl.mem_flags
        
        self.device_u = cl.Buffer(self.ctx, mf.READ_ONLY, size = self.shape[0] * np.dtype(np.float32).itemsize)
        self.device_v = cl.Buffer(self.ctx, mf.WRITE_ONLY, size = self.shape[0] * np.dtype(np.float32).itemsize)

        self.program = cl.Program(self.ctx, """
        __kernel void stencil_kernel(
            __global const double *u,
            __global double *v
        )
        {
            int gid = get_global_id(0);
 
            v[gid] = 0.5 * u[gid]; 
        }
        """).build()

    
    def _matvec(self, u):
        """
        """
        
        self.dtype = u.dtype
        
        cl.enqueue_copy(self.queue, self.device_u, u)

        self.program.stencil_kernel(self.queue, \
                                   (self.shape[0],), \
                                   (1,), \
                                   self.device_u, \
                                   self.device_v)

        v = np.zeros((self.shape[0]), dtype=np.float32)

        cl.enqueue_copy(self.queue, v, self.device_v)
            
        return v

In [308]:
# Create an instance of StencilOperatorOpenCL.

N = 4

stencil = StencilOperatorOpenCL(N)

In [309]:
# Test the instance.

u = np.full((N * N), 4, dtype=np.float32)

stencil.matvec(u)

array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
      dtype=float32)

In [313]:
# Use Conjugate Iteration.

N = 4

u = np.full((N * N), 4, dtype=np.float32)

b = np.ones((N * N), dtype=np.float32)

def cg_callback(xk):
    print(xk.reshape(N, N))

solution, info = cg(StencilOperatorOpenCL(N), b, x0 = u, callback = cg_callback)

[[2. 2. 2. 2.]
 [2. 2. 2. 2.]
 [2. 2. 2. 2.]
 [2. 2. 2. 2.]]


In [316]:
solution.reshape(N, N)

array([[2., 2., 2., 2.],
       [2., 2., 2., 2.],
       [2., 2., 2., 2.],
       [2., 2., 2., 2.]], dtype=float32)